# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

## Implementations

In [ ]:
from collections import Counter

In [ ]:
class AbstractSpellCorrector:
    """
    Abstarct class for any spell corrector
    """
    def __init__(self, sentences: list[list[str]]) -> None:
        """
        Train spell corrector by providing sentences with correct grammar

        Parameters:
            sentences (list[list[str]]): List of sentences
        """
        raise Exception("Not implemented")

    def correct_sentence(self, sentence: list[str]) -> list[str]:
        """
        Correct sentence with errors

        Parameters:
            sentence (list[str]): List of words
        Returns:
            (list[str]): List of corrected words
        """
        raise Exception("Not implemented")

    def correct_text(self, text: list[list[str]]) -> list[list[str]]:
        """
        Corrects the whole text

        Parameters:
            text (list[list[str]]): The text to correct
        Returns:
            (list[list[str]]): Corrected text
        """
        corrected_text = []
        for sentence in text:
            corrected_text.append(self.correct_sentence(sentence))
        return corrected_text


### Norvig's spell corrector

In [ ]:
class NorvigSpellCorrector(AbstractSpellCorrector):
    """
    Implementation of Norvig's spell corrector
    """
    def __init__(self, sentences: list[list[str]]) -> None:
        text = []
        for sentence in sentences:
            for word in sentence:
                text.append(word)
        self.word_counter = Counter(text)
        self.N = sum(self.word_counter.values())

    def correct_sentence(self, sentence: list[str]) -> list[str]:
        corrected = []
        for word in sentence:
            corrected.append(self.__correct_word(word))
        return corrected

    def __get_word_probability(self, word: str) -> float:
        """
        Get probability of `word`.

        Parameters:
            word (str): The word
        Returns
            (float): Probability of occuring of these word in the sentence
        """
        return self.word_counter[word] / self.N

    def __correct_word(self, word: str) -> str:
        """
        Correct word by suggesting most probable spelling correction for word

        Parameters:
            word (str): The word to correct
        Returns:
            (str): Corrected word
        """
        return max(self._generate_candidates(word), key=self.__get_word_probability)

    def _generate_candidates(self, word: str) -> set[str]:
        """
        Generate possible spelling corrections for word

        Parameters:
            word (str): Word for suggesting corrections
        Returns:
            (set[str]): Correction candidates
        """
        return (self._extract_known([word]) or self._extract_known(self._get_edits1(word)) or self._extract_known(self._get_edits2(word)) or [word])

    def _extract_known(self, words: list[str]|set[str]) -> set[str]:
        """
        The subset of `words` that appear in the dictionary of WORDS.

        Parameters:
            words (list[str] | set[str]): Collection of words
        Returns:
            (set[str]): Set of only known worlds
        """
        return set(word for word in words if word in self.word_counter)

    def _get_edits1(self, word: str) -> set[str]:
        """
        Get all edits that are one edit away from `word`.

        Parameters:
            word (str): The word to edit
        Returns:
            (set[str]): All words edits
        """
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def _get_edits2(self, word: str) -> list[str]:
        """
        Get all edits that are two edits away from `word`.

        Parameters:
            word (str): The word to edit
        Returns:
            (set[str]): All words edits
        """
        return [e2 for e1 in self._get_edits1(word) for e2 in self._get_edits1(e1)]

### Custom spell corrector

In [ ]:
from math import log

class CustomSpellCorrector(NorvigSpellCorrector):
    """
    My upgraded Norvig's spell checker.
    """
    def __init__(self, sentences: list[list[str]], n:int = 3, penalty1: int = 0.05, penalty2: int = 0.1) -> None:
        """
        Train spell corrector by providing sentences with correct grammar

        Parameters:
            sentences (list[list[str]]): List of sentences
            n (int): Which n-grams to consider in language model (if n = 2 then unigrams and bigrams will be taken into account)
            penalty1 (int): Penalty for 1 distance edits of words
            penalty2 (int): Penalty for 2 distance edits of words
        """
        super().__init__(sentences)

        self.n = n
        self.penalty1 = penalty1
        self.penalty2 = penalty2

        ngrams = []
        for sentence in sentences:
            for k in range(2, n+1):
                for i in range(0, len(sentence)-k+2):
                    if i < len(sentence)-k+1:
                        ngram = sentence[i:i+k]
                        ngrams.append(" ".join(ngram))

                    ngram_without_last = sentence[i:i+k-1]
                    ngrams.append(" ".join(ngram_without_last))
        self.ngram_counter = Counter(ngrams)
        self.vocab_size = len(Counter(self.word_counter).keys())

    def correct_sentence(self, sentence: list[str]) -> list[str]:
        corrected = []
        for i in range(len(sentence)):
            corrected.append(self.__correct_word(i, sentence))
        return corrected

    def change_penalties(self, penalty1: int, penalty2: int) -> None:
        """
        Change penalties for 1 distance and 2 distance edits

        Parameters:
            penalty1 (int): Penalty for 1 distance edits
            penalty2 (int): Penalty for 2 distance edits
        """
        self.penalty1 = penalty1
        self.penalty2 = penalty2

    def __correct_word(self, position: int, sentence: list[str]) -> str:
        word = sentence[position]
        candidates = [(self.__get_sentence_probability(sentence), word)]

        for candidate in self._extract_known(self._get_edits1(word)):
            corrected_sentence = sentence.copy()
            corrected_sentence[position] = candidate
            candidates.append((self.__get_sentence_probability(corrected_sentence) - self.penalty1, candidate))

        for candidate in self._extract_known(self._get_edits2(word)):
            corrected_sentence = sentence.copy()
            corrected_sentence[position] = candidate
            candidates.append((self.__get_sentence_probability(corrected_sentence) - self.penalty2, candidate))

        return max(candidates, key=lambda x: x[0])[1]

    def _generate_candidates(self, word: str) -> set[str]:
        return (self._extract_known(self._get_edits1(word)) or self._extract_known(self._get_edits2(word)) or [word])

    def __get_sentence_probability(self, sentence: list[str]) -> float:
        """
        Get probability of whole sentence

        Parameters:
            sentence (list[str]): The sentence
        Returns:
            (float): Probability of provided sentence to occure in the text
        """
        probability = 0.0
        for k in range(2, self.n):
            for i in range(0, len(sentence)-k+1):
                ngram = " ".join(sentence[i:i+k])
                ngram_without_last = " ".join(sentence[i:i+k-1])
                probability += log((self.ngram_counter[ngram]+1)/(self.vocab_size + self.ngram_counter[ngram_without_last]))

        for word in sentence:
            probability += log((self.word_counter[word]+1)/(self.N + self.vocab_size))
        return probability

temp = CustomSpellCorrector([["hello", "test", "version", "hello"]])
print(f"VOCAB SIZE - {temp.vocab_size}")
print(temp.correct_sentence(["hello", "hello", "test", "version"]))

VOCAB SIZE - 3
['hello', 'hello', 'test', 'version']


### Train correctors on a dataset

In [ ]:
# Download train dataset
%%capture
!wget https://downloads.wortschatz-leipzig.de/corpora/eng-simple_wikipedia_2021_300K.tar.gz
!tar -xvzf eng-simple_wikipedia_2021_300K.tar.gz

In [ ]:
# Download train dataset
%%capture
!wget https://downloads.wortschatz-leipzig.de/corpora/eng_news_2023_300K.tar.gz
!tar -xvzf eng_news_2023_300K.tar.gz

In [ ]:
import pandas as pd

wikipedia_df = pd.read_csv("eng-simple_wikipedia_2021_300K/eng-simple_wikipedia_2021_300K-sentences.txt", sep='\t', header=None, names=["id", "sentence"])
news_df = pd.read_csv("eng_news_2023_300K/eng_news_2023_300K-sentences.txt", sep='\t', header=None, names=["id", "sentence"])

In [ ]:
from tqdm import tqdm
import re

def load_to_corpus(df: pd.DataFrame, corpus: list[str]) -> None:
    """
    Load dataset with sentence's row to a text corpus

    Parameters:
        df (DataFrame): DataFrame that has sentence row
        corpus (list[str]): Text corpus to extend
    """
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        sentence = row["sentence"]
        words = re.findall(r'\w+', sentence.lower())
        corpus.append(words)


train_corpus = []
load_to_corpus(wikipedia_df, train_corpus)
load_to_corpus(news_df, train_corpus)

train_corpus[1]

100%|██████████| 249986/249986 [00:18<00:00, 13244.98it/s]


['100', 'coin', 'struck', 'only', 'in', 'proof']

In [ ]:
norvig_spell_corrector = NorvigSpellCorrector(train_corpus)
custom_spell_corrector = CustomSpellCorrector(train_corpus, penalty1= 4.9, penalty2 = 12.1)

In [ ]:
# Sanity check
check_text = [
                ["bleu", "lilte", "boy", "really", "loves", "appls"],
                ["he", "ws", "scared", "uf", "snkes"]
              ]

print(f"Norvig's solution: {norvig_spell_corrector.correct_text(check_text)}")
print(f"Custom spell corrector: {custom_spell_corrector.correct_text(check_text)}")

Norvig's solution: [['bleu', 'lite', 'boy', 'really', 'loves', 'apple'], ['he', 'ws', 'scared', 'uf', 'snakes']]
Custom spell corrector: [['blue', 'little', 'boy', 'really', 'loves', 'apple'], ['he', 'was', 'scared', 'of', 'snakes']]


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### Decisions
#### <font color='yellow'> Implementing language model with trigrams</font>
Reason: I decided to implement a language model using trigrams for improved accuracy without significant slowdown in both training and inference for spell checking.

#### <font color='yellow'> Training on Wikipedia and News articles </font>
Reason: I utilized text corpora from Wikipedia and news articles for training purposes. These diverse sources cover a wide range of topics and domains, making them ideal for training a general-purpose spell checker. Additionally, the texts are meticulously reviewed by correctors, ensuring they are error-free and devoid of misspellings.

#### <font color='yellow'> Assigning specific penalties to edits  </font>
Reason: I have incorporated specific penalties to reduce the occurrence of false corrections. In my strategy, I have assigned a penalty of 4.9 for single edits and 12.1 for two consecutive edits of a word. After conducting various experiments with different penalty values, these particular settings have proven to deliver the most high fix rate.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

**I did not generate test set but used one from Hugging face https://huggingface.co/datasets/vishnun/SpellGram for more fair results**

In [ ]:
# Downloading test set
!wget https://huggingface.co/datasets/vishnun/SpellGram/raw/main/train.csv

--2024-03-06 20:12:34--  https://huggingface.co/datasets/vishnun/SpellGram/raw/main/train.csv
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.55, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4489429 (4.3M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   4.28M  --.-KB/s    in 0.1s    

2024-03-06 20:12:34 (43.8 MB/s) - ‘train.csv’ saved [4489429/4489429]



In [ ]:
test_df = pd.read_csv("train.csv")
test_df.head(5)

,source,target
0,rate the silent upeaker four out oe 6,rate the silent speaker four out of 6
1,please find me tqe gork tqe bfrning sorld,please find me the work the burning world
2,three friendl afe relaxing uround the tsble,three friends are relaxing around the table
3,what dm they want,what do they want
4,man in tan aat working with stones,man in tan hat working with stones


In [ ]:
def evaluate_spell_corrector(test_df: pd.DataFrame, spell_corrector: AbstractSpellCorrector):
    """
    Evaluates provided spell corrector by `fix-rate` and `broken` metrics
    """
    true_fixed_errors = 0
    false_fixed_errors = 0
    errors = 0
    no_errors = 0

    for index, sample in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        source = re.findall(r'\w+', sample["source"].lower())
        target = re.findall(r'\w+', sample["target"].lower())

        corrected = spell_corrector.correct_sentence(source)
        for i in range(len(source)):
            if source[i] != target[i]:
                errors += 1
                if corrected[i] == target[i]:
                    true_fixed_errors += 1
            else:
                no_errors += 1
                if corrected[i] != source[i]:
                    false_fixed_errors += 1

    return true_fixed_errors / errors, false_fixed_errors / no_errors

n_subset = 2000 # How much samples e will take to test

fix_rate, broken = evaluate_spell_corrector(test_df.head(n_subset), norvig_spell_corrector)
print()
print("NORVIG's SPELL CORRECTOR:")
print(f"FIX RATE: {fix_rate}")
print(f"BROKEN: {broken}")

fix_rate, broken = evaluate_spell_corrector(test_df.head(n_subset), custom_spell_corrector)
print()
print("MY CUSTOM SPELL CORRECTOR:")
print(f"FIX RATE: {fix_rate}")
print(f"BROKEN: {broken}")

100%|██████████| 2000/2000 [02:18<00:00, 14.42it/s]



NORVIG's SPELL CORRECTOR:
FIX RATE: 0.4969631236442516
BROKEN: 0.01782602868248224


100%|██████████| 2000/2000 [29:44<00:00,  1.12it/s]


MY CUSTOM SPELL CORRECTOR:
FIX RATE: 0.7067245119305857
BROKEN: 0.04268864763436537


<font color="lime">As you can see, my solution works 20% better for fix rate, while the "broken" metric has increased by only about 2.5%</font>

---

